### Gathering sentence data for each word

In [1184]:
word = 'francais' # edit this per word 
permutations = [' francais ',' Francais ',' FRANCAIS ',] # add in list of permutations to account for plural, gender and spelling variations
language = 'fr' # add the language you want sentence data for

In [ ]:
# creating a string of permutations for regex
permutations_string = '|'.join(permutations)
permutations_string = permutations_string.strip('|')

In [1185]:
# initialising the file path variable according to which language you want sentence data for 
if language == 'en':
    folder_name = 'EN_Data/'
elif language == 'fr':
    folder_name = 'FR_Data/'

In [ ]:
# getting the list of data files, a sub-set of OSCAR corpus 
from os import listdir
files = listdir(path = folder_name)

['fr_meta_part_102.jsonl', 'fr_meta_part_100.jsonl', '.DS_Store', 'fr_meta_part_101.jsonl', 'fr_meta_part_103.jsonl', 'fr_meta_part_10.jsonl', 'fr_meta_part_1.jsonl']


In [1192]:
import re
import json
for name in files[:2]: # edit the index of the files if the word is common to reduce runtime
    with open(folder_name+name,'r') as json_file:
        try:
            result = [json.loads(jline) for jline in json_file]
        except UnicodeDecodeError:
             continue
    longlist = []
    for item in result:
            for p in permutations:
                if p in item['content']:
                    longlist+=[item['content']]
    pattern = r'(?<!,)[^.]{,50}(?:'+permutations_string+r')[^.]{,50}' # regex to find variations of target word
    punct = r'[",#$%&\'\(\)\*\+\/:;<=>@\[\\\]^`{\|}~»«]+'
    spaces = r'[\.!\?\-_]+'
    shortertext = []
    for line in longlist:
        line = str(line)
        mo = re.findall(pattern,line) # finding all matches for regex
        if mo != None:
            for item in mo: # making the phrases lowercase and removing punctuation
                item = item.lower()
                item = re.sub(punct,'',item)
                item = re.sub(spaces,' ',item) 
                sentence_list = item.split()
                if len(sentence_list)>50: # truncating the phrase to make it more likely to fit in the model
                    shorter = sentence_list[:50]
                    short = ' '.join([str(element) for element in shorter])
                    shortertext +=[short]
                else:
                    shortertext+=[item]
    no_doubles = list(dict.fromkeys(shortertext)) # removing duplicates
    no_doubles_shorter = no_doubles[:1000]
    with open(word+'_'+language+'_unique.txt','a') as unique:
        for item in no_doubles_shorter:
                unique.write(item)

In [1193]:
 # checking length and initiating variable of sentences
with open(word+'_'+language+'_unique.txt','r') as final_list:
    sentences = final_list.readlines()
    print(len(sentences))

1097
